In [ ]:
import pandas as pd

def clean_data(df):
    #Drop columns: 'id', 'url' and 25 other columns
    df = df.drop(columns=['id', 'url', 'province', 'monthlyCost', 'hasAttic', 'hasBasement', 'hasDressingRoom', 'diningRoomSurface', 'hasDiningRoom', 'roomCount', 'streetFacadeWidth', 'hasLift', 'hasHeatPump', 'hasPhotovoltaicPanels', 'hasThermicPanels', 'kitchenSurface', 'hasLivingRoom', 'livingRoomSurface', 'hasBalcony', 'gardenOrientation', 'hasAirConditioning', 'hasArmoredDoor', 'hasVisiophone', 'hasOffice', 'terraceOrientation', 'accessibleDisabledPeople', 'Unnamed: 0'])
    # Drop 'APARTMENT_GROUP' and 'HOUSE_GROUPR values in type column
    df = df[~df['type'].isin(['APARTMENT_GROUP', 'HOUSE_GROUP'])]
    # Drop 'APARTMENT_BLOCK' in subtype column
    df = df[~df['subtype'].isin(['APARTMENT_BLOCK'])]
    # 1. Calculer la médiane du nombre de chambres par sous-type
    # (avant toute imputation)
    median_by_subtype = df.groupby('subtype')['bedroomCount'].median()
    print("Médianes par type de bien:")
    print(median_by_subtype)
    # 2. Pour chaque sous-type, remplacer les valeurs manquantes par la médiane correspondante
    for subtype in df['subtype'].unique():
        # Créer un masque qui identifie les lignes où:
        # - Le sous-type correspond à celui qu'on traite actuellement
        # - ET la valeur de bedroomCount est manquante
        mask = (df['subtype'] == subtype) & (df['bedroomCount'].isna())
        # Remplacer ces valeurs par la médiane du sous-type
        df.loc[mask, 'bedroomCount'] = median_by_subtype[subtype]
    # 3. Vérifier s'il reste des valeurs manquantes
    remaining_missing = df['bedroomCount'].isna().sum()
    print(f"Valeurs manquantes restantes: {remaining_missing}")
    # 4. Si certains sous-types n'ont pas de médiane (tous NaN),
    # utiliser la médiane globale pour les valeurs encore manquantes
    if remaining_missing > 0:
        global_median = df['bedroomCount'].dropna().median()
        df['bedroomCount'].fillna(global_median, inplace=True)
        print(f"Médiane globale utilisée pour les cas restants: {global_median}")
    # Replace missing values with 1 in column: 'bathroomCount'
    df = df.fillna({'bathroomCount': 1})
    # Replace missing values with the median of each column in: 'habitableSurface'
    df = df.fillna({'habitableSurface': df['habitableSurface'].median()})
    # Replace missing values with "UNKNOWN" in column: 'buildingCondition'
    df = df.fillna({'buildingCondition': "UNKNOWN"})
    # Replace missing values with 0 in column: 'buildingConstructionYear'
    df = df.fillna({'buildingConstructionYear': 0})
    # Change column type to int64 for column: 'buildingConstructionYear'
    df = df.astype({'buildingConstructionYear': 'int64'})
    # Drop columns: 'floorCount', 'landSurface' and 3 other columns
    df = df.drop(columns=['floorCount', 'landSurface', 'parkingCountIndoor', 'parkingCountOutdoor', 'terraceSurface'])
    # Replace missing values with 2 in column: 'facedeCount'
    df = df.fillna({'facedeCount': 2})
    # Replace missing values with the most common value of each column in: 'floodZoneType'
    df = df.fillna({'floodZoneType': df['floodZoneType'].mode()[0]})
    # Replace missing values with the most common value of each column in: 'heatingType'
    df = df.fillna({'heatingType': df['heatingType'].mode()[0]})
    # Replace missing values with the most common value of each column in: 'kitchenType'
    df = df.fillna({'kitchenType': df['kitchenType'].mode()[0]})
    # Change column type to string for column: 'hasGarden'
    df = df.astype({'hasGarden': 'string'})
    # Replace all instances of "True" with "1" in column: 'hasGarden'
    df['hasGarden'] = df['hasGarden'].str.replace("True", "1", case=False, regex=False)
    # One-hot encode column: 'hasGarden'
    insert_loc = df.columns.get_loc('hasGarden')
    df = pd.concat([df.iloc[:,:insert_loc], pd.get_dummies(df.loc[:, ['hasGarden']]), df.iloc[:,insert_loc+1:]], axis=1)
    # Drop column: 'gardenSurface'
    df = df.drop(columns=['gardenSurface'])
    # Replace missing values with 1 in column: 'toiletCount'
    df = df.fillna({'toiletCount': 1})
    # Change column type to string for column: 'hasSwimmingPool'
    df = df.astype({'hasSwimmingPool': 'string'})
    # One-hot encode column: 'hasSwimmingPool'
    insert_loc = df.columns.get_loc('hasSwimmingPool')
    df = pd.concat([df.iloc[:,:insert_loc], pd.get_dummies(df.loc[:, ['hasSwimmingPool']]), df.iloc[:,insert_loc+1:]], axis=1)
    # Change column type to string for column: 'hasFireplace'
    df = df.astype({'hasFireplace': 'string'})
    # One-hot encode column: 'hasFireplace'
    insert_loc = df.columns.get_loc('hasFireplace')
    df = pd.concat([df.iloc[:,:insert_loc], pd.get_dummies(df.loc[:, ['hasFireplace']]), df.iloc[:,insert_loc+1:]], axis=1)
    # Change column type to string for column: 'hasTerrace'
    df = df.astype({'hasTerrace': 'string'})
    # One-hot encode column: 'hasTerrace'
    insert_loc = df.columns.get_loc('hasTerrace')
    df = pd.concat([df.iloc[:,:insert_loc], pd.get_dummies(df.loc[:, ['hasTerrace']]), df.iloc[:,insert_loc+1:]], axis=1)
    # Drop rows with missing data in column: 'epcScore'
    df = df.dropna(subset=['epcScore'])
    # Drop rows with missing data in column: 'price'
    df = df.dropna(subset=['price'])
    # Change column type to int64 for column: 'bedroomCount'
    df = df.astype({'bedroomCount': 'int64'})
    # Change column type to int64 for column: 'bathroomCount'
    df = df.astype({'bathroomCount': 'int64'})
    # Change column type to int64 for column: 'facedeCount'
    df = df.astype({'facedeCount': 'int64'})
    # Change column type to int64 for column: 'toiletCount'
    df = df.astype({'toiletCount': 'int64'})
    # Filter rows based on column: 'bedroomCount'
    df = df[df['bedroomCount'] <= 35]
    # Filter rows based on column: 'bathroomCount'
    df = df[df['bathroomCount'] <= 30]
    # Filter rows based on column: 'habitableSurface'
    df = df[df['habitableSurface'] <= 65000]
    # Filter rows based on column: 'buildingConstructionYear'
    df = df[df['buildingConstructionYear'] <= 2025]
    # Filter rows based on column: 'facedeCount'
    df = df[df['facedeCount'] <= 5]
    # Filter rows based on column: 'toiletCount'
    df = df[df['toiletCount'] <= 32]
    # Filter rows based on column: 'epcScore'
    df = df[(df['epcScore'] != "X") & (~df['epcScore'].str.contains("_", regex=False, na=False, case=False))]
    # Sort by column: 'price' (descending)
    df = df.sort_values(['price'], ascending=[False])
    return df

# Loaded variable 'df' from URI: g:\Mon Drive\BeCode\Bootcamp AI\Immo Eliza project\immo-eliza-machine-learning\immo-eliza-machine-learning-Linwe-e\data\Kangaroo.csv
df = pd.read_csv(r'g:\Mon Drive\BeCode\Bootcamp AI\Immo Eliza project\immo-eliza-machine-learning\immo-eliza-machine-learning-Linwe-e\data\Kangaroo.csv')

df_clean = clean_data(df.copy())
df_clean.head()